In [5]:
import pandas as pd
import sys
sys.path.append('/home/handeully/')
import bigquery_etl as bq
# GCP Env.
import google.auth
from google.cloud import bigquery
from googletrans import Translator
from google_trans_new import google_translator

# Coding Env.
import warnings
credentials, project_id = google.auth.default(
    scopes=["https://www.googleapis.com/auth/cloud-platform"]
)

client = bigquery.Client(credentials=credentials, project=project_id )
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 1000)
warnings.filterwarnings("ignore", category=DeprecationWarning)

# sql 
review_sql = f'''
                WITH base_table AS (SELECT A.*,B.prdct_ctgry_4_5,B.brand,RANK() OVER(partition by A.reviewId order by crawlTime desc) as rank
                                    FROM `market-analysis-project-91130`.crwl.amz_rvw_all A 
                                    LEFT JOIN `market-analysis-project-91130`.meta.crwl_amz_pdt_mst_all B ON A.asin =B.asin)
                SELECT  reviewId,
                        region,		
                        lower(brand)as brand,
                        asin,
                        date(date)as date,
                        helpful_vote,
                        initialTime,
                        rating,
                        reviewer_url,
                        product_option,
                        title,
                        verified_purchase,
                        crawlTime,
                        review_text,
                        prdct_ctgry_4_5
                FROM base_table
                WHERE prdct_ctgry_4_5 is not null and rank = 1
                AND reviewId not in (select reviewId
                                          from (
                                                 SELECT reviewId,count(DISTINCT prdct_ctgry_4_5)--A.*,B.prdct_ctgry_4_5,RANK() OVER(partition by A.reviewId order by crawlTime desc) as rank
                                                 FROM base_table
                                                 GROUP BY reviewId 
                                                 HAVING count(DISTINCT prdct_ctgry_4_5) > 1
                                             )
                                     )		
                AND substring(date,1,4) >= '2020'
                AND rating in(1,2,5)
                ORDER BY DATE ASC
             '''


df                      =  bq.select_query(review_sql)

print(df.shape)

(935519, 15)
